In [25]:
import os

from datetime import datetime
from pathlib import Path
from pprint import pprint
from subprocess import run, DEVNULL


In [36]:
def lvs(spice1: Path, cell1: str, spice2: Path, cell2: str, DEBUG: bool=False):
    logfile = Path(f"lvs-{datetime.now().strftime('%d-%m-%Y-%H-%M-%S')}.log")
    techfile = Path(os.environ["PDK_ROOT"]) / os.environ["PDK"] / "libs.tech" / "netgen" / "gf180mcuC_setup.tcl"

    if logfile.exists():
        print("Logfile is going to be overwritten")
        return

    if not techfile.exists():
        print("Netgen tech file not existent. Abort")
        return
    
    # Add cell name
    cmd = [
        "netgen",
        "-batch",
        "lvs",
        f"\"{spice1} {cell1}\"",
        f"\"{spice2} {cell2}\"",
        str(techfile),
        str(logfile),
    ]

    if DEBUG: 
        print("CMD: " + " ".join(cmd))

    return cmd
    out = run(cmd, capture_output=True)

    return {
        "returncode": out.returncode,
        "stdout": out.stdout.decode("utf-8"),
        "stderr": out.stderr.decode("utf-8"),
    }

#spice1 = Path(f"/home/{os.environ['USER']}/.xschem/simulations/ldo.spice")
spice1 = Path("/workspaces") / "DC23-LTC2-LDO" / "LDO" / "test" / "transistor.spice"
spice2 = spice1

cell1 = "transistor"
cell2 = "transistor"

DEBUG=False
#DEBUG=True
out = lvs(spice1, cell1, spice2, cell2, DEBUG)

# print(out["stdout"])
# print(out["stderr"])
print(" ".join(out))

netgen -batch lvs "/workspaces/DC23-LTC2-LDO/LDO/test/transistor.spice transistor" "/workspaces/DC23-LTC2-LDO/LDO/test/transistor.spice transistor" /home/designer/.volare/gf180mcuC/libs.tech/netgen/gf180mcuC_setup.tcl lvs-14-11-2023-01-38-04.log


In [37]:
def magic(gdspath: Path, DEBUG=False):
    logfile = Path(f"magic-{datetime.now().strftime('%d-%m-%Y-%H-%M-%S')}.log")
    techfile = Path(os.environ["PDK_ROOT"]) / os.environ["PDK"] / "libs.tech" / "magic" / "gf180mcuC.magicrc"

    if logfile.exists():
        print("Logfile is going to be overwritten")
        return

    if not techfile.exists():
        print("Netgen tech file not existent. Abort")
        return

    # En alguna parte hay que crear una celda top
    # Y vincular las subceldas a este top.
    magic_cmd = [
        "<<EOF",
        "drc off",
        f"gds read {gdspath}",
        "getcell transistor",
        "load transistor",
        "box 0 0 0 0",
        #"findbox zoom",
        #"expand",
        #"extract cell transistor",    # Innecesario
        "extract",
        "ext2spice lvs",
        "ext2spice",
        "exit",
        "EOF",
    ]
    cmd = [
        "magic",
        "-dnull",
        "-noconsole",
        "-rcfile",
        str(techfile),
        "\n".join(magic_cmd),
    ]

    print(cmd)

    if DEBUG:
        print("CMD: " + " ".join(cmd))

    #return cmd

    out = run(cmd, capture_output=True, stdin=DEVNULL)

    return {
        "returncode": out.returncode,
        "stdout": out.stdout.decode("utf-8"),
        "stderr": out.stderr.decode("utf-8"),
    }

gdspath= Path("/workspaces") / "DC23-LTC2-LDO" / "LDO" / "test" / "transistor.gds"

DEBUG=True
out = magic(gdspath, DEBUG=DEBUG)

print(out["stdout"])
print(out["stderr"])

['magic', '-dnull', '-noconsole', '-rcfile', '/home/designer/.volare/gf180mcuC/libs.tech/magic/gf180mcuC.magicrc', '<<EOF\ngds read /workspaces/DC23-LTC2-LDO/LDO/test/transistor.gds\ngetcell transistor\nload transistor\nbox 0 0 0 0\nextract\next2spice lvs\next2spice\nexit\nEOF']
CMD: magic -dnull -noconsole -rcfile /home/designer/.volare/gf180mcuC/libs.tech/magic/gf180mcuC.magicrc <<EOF
gds read /workspaces/DC23-LTC2-LDO/LDO/test/transistor.gds
getcell transistor
load transistor
box 0 0 0 0
extract
ext2spice lvs
ext2spice
exit
EOF

Magic 8.3 revision 445 - Compiled on Thu Nov  9 15:31:55 UTC 2023.
Starting magic under Tcl interpreter
Using the terminal as the console.
Using NULL graphics device.
Processing system .magicrc file
Sourcing design .magicrc for technology gf180mcuC ...
10 Magic internal units = 1 Lambda
Input style import: scaleFactor=10, multiplier=2
The following types are not handled by extraction and will be treated as non-electrical types:
    obsactive mvobsactive fill